#### CNN
- 학습시간이 줄었음
- 2차원 데이터를 그대로 추출
- 특성 데이터 추출

In [73]:
# module loading
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Input, Dense
from tensorflow.keras.datasets.mnist import load_data

import numpy as np
import matplotlib.pyplot as plt

#### 데이터 준비
- setting dimension shape
- scaling => 0.0 ~ 1.0

[1] 데이터 로딩

In [74]:
(X_train, y_train), (X_test, y_test) = load_data()

[2] 차원 처리
- Conv => ( row, col, channel ) 또는 ( channel, row, col )

In [75]:
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [76]:
# Use np.reshape
X_train_in = np.reshape(X_train, (-1, 28, 28, 1))
X_test_in = np.reshape(X_test, (-1, 28, 28, 1))

In [77]:
X_train_in.shape, X_test_in.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [78]:
# TF 텐서 형태(shape) 변형 관련
# tf.squeeze()        # 1로 설정된 행 / 열 / 차원을 제거
# tf.expand_dims()    # 지정된 위치에 행 / 열 / 차원을 1 추가

In [79]:
X_train_in2 = tf.expand_dims(X_train, -1)
X_train_in3 = tf.expand_dims(X_train, 0)    # 0번 인덱스 자리에 1 추가
X_train_in4 = tf.expand_dims(X_train_in3, 0)

In [80]:
X_train_in2.shape, X_train_in3.shape, X_train_in4.shape

(TensorShape([60000, 28, 28, 1]),
 TensorShape([1, 60000, 28, 28]),
 TensorShape([1, 1, 60000, 28, 28]))

In [81]:
# 1이 있는 축 삭제
# 만약 1이 여러 개라면 인덱스를 지정해 원하는 축만 삭제
X_train_in22 = tf.squeeze(X_train_in2)
X_train_in33 = tf.squeeze(X_train_in3)
X_train_in44 = tf.squeeze(X_train_in4, 1)   # 1번 인덱스의 축 삭제

In [82]:
X_train_in22.shape, X_train_in33.shape, X_train_in44.shape

(TensorShape([60000, 28, 28]),
 TensorShape([60000, 28, 28]),
 TensorShape([1, 60000, 28, 28]))

[3] 전처리

In [83]:
# X_train_sc = tf.cast(X_train, tf.float32) / 255

In [84]:
X_train_in = X_train_in / X_train_in.max()
X_test_in = X_test_in / X_test_in.max()

In [85]:
(X_train_in.min(), X_train_in.max()), (X_test_in.min(), X_test_in.max())

((0.0, 1.0), (0.0, 1.0))

#### Mode 설계

[1] Sequential 방식
- 특징맵 추출 파트
    * 입력층 => Input
    * 은닉층 => Conv2D
    * 은닉층 => Conv2D
    * 은닉층 => Pooling
<br>
<br>
- 분류 파트
    * 1차원 변환 층 ==> Flatten
    * 출력층 ==> Dense()

[1-1] 모델 설계

In [86]:
model = Sequential()
model.add(Input(shape=(28, 28, 1)))
model.add(Conv2D(20, (3, 3), activation = 'relu'))
model.add(Conv2D(20, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (3, 3)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

In [87]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 20)        200       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 20)        3620      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 20)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                12810     
Total params: 16,630
Trainable params: 16,630
Non-trainable params: 0
_________________________________________________________________
